In [1]:
import os
import pandas as pd

import sklearn.ensemble as ske

from bounos.Analyses.Weight import summed_outliers_per_weight

def categorise_dataframe(df):
    # Categories work better as indexes
    for obj_key in df.keys()[df.dtypes == object]:
        try:
            df[obj_key] = df[obj_key].astype('category')
        except TypeError:
            print("Couldn't categorise {}".format(obj_key))
            pass
    return df

def non_zero_rows(df):
    return df[~(df==0).all(axis=1)]

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


The questions we want to answer are:
1. What metrics differentiate between what behaviours
2. Do these metrics cross over domains (i.e. comms impacting behaviour etc)

To answer these questions we first have to manipulate the raw dataframe to be weight-indexed with behaviour(`var`) keys on the perspective from the observer to the (potential) attacker in a particular run (summed across the timespace)

While this analysis simply sums both the upper and lower outliers, **this needs extended/readdressed**

# IMPORTANT 
The July 3rd Simulation Run had a small mistake where the Ran

In [24]:
observer = 'Bravo'
target = 'Alfa'
n_nodes = 6
n_metrics = 9

results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-07-03-16-45-26"
results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Comparison-2015-07-20-17-47-53"
shared_h5_path = '/dev/shm/shared.h5'
#Something smells
shared_h5_path = '/home/bolster/src/aietes/results/shared.bkup.h5'


with pd.get_store(shared_h5_path) as store:
    joined_target_weights = store.get('joined_target_weights')
    joined_feats = store.get('joined_feats')
    comms_only_feats = store.get('comms_only_feats')
    phys_only_feats = store.get('phys_only_feats')
    comms_only_weights = store.get('comms_only_weights')
    phys_only_weights = store.get('phys_only_weights')

joined_feat_weights = categorise_dataframe(non_zero_rows(joined_feats).T)
comms_feat_weights = categorise_dataframe(non_zero_rows(comms_only_feats).T)
phys_feat_weights = categorise_dataframe(non_zero_rows(phys_only_feats).T)

In [32]:
with pd.get_store(results_path + "/outliers.bkup.h5") as store:
    weight_df = store.get(store.keys()[0])
weight_df

target,var,t,run,observer,range,Alfa,Bravo,Charlie,Delta,Echo,Foxtrot,ADelay,ARXP,ATXP,RXThroughput,PLR,TXThroughput,INDD,INHD,Speed
0,CombinedBadMouthingPowerControl,2,0,Bravo,lower,NaN,0.039826,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,CombinedBadMouthingPowerControl,4,0,Bravo,lower,NaN,NaN,NaN,0.123390,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,CombinedBadMouthingPowerControl,5,0,Bravo,lower,NaN,NaN,NaN,0.123751,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,CombinedBadMouthingPowerControl,7,0,Bravo,lower,NaN,0.026199,NaN,0.123658,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,CombinedBadMouthingPowerControl,8,0,Bravo,lower,NaN,0.040296,NaN,0.120131,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,CombinedBadMouthingPowerControl,10,0,Bravo,lower,NaN,NaN,NaN,0.062902,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
6,CombinedBadMouthingPowerControl,15,0,Bravo,lower,NaN,NaN,NaN,0.123751,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
7,CombinedBadMouthingPowerControl,16,0,Bravo,lower,NaN,0.037748,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
8,CombinedBadMouthingPowerControl,17,0,Bravo,lower,NaN,0.039706,NaN,0.123751,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
9,CombinedBadMouthingPowerControl,18,0,Bravo,lower,NaN,NaN,NaN,0.123751,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [33]:
weight_df = joined_target_weights

In [34]:
target_var = 'CombinedBadMouthingPowerControl'
df = weight_df[target_var].reset_index()
data = df.drop(target_var, axis=1).values
labels = df[target_var].values
etr = ske.ExtraTreesRegressor(n_jobs=4, n_estimators=512)
rtr = ske.RandomForestRegressor(n_jobs=4, n_estimators=512)

In [35]:
linr = linear_model.LinearRegression()
for reg in [etr,rtr,linr]:
    scores = cross_validation.cross_val_score(reg, data, labels, scoring='mean_squared_error', n_jobs=4)
    print scores, sp.stats.describe(scores)

[-3.383 -0.912 -1.263] DescribeResult(nobs=3, minmax=(-3.3833940697877742, -0.91164012364138525), mean=-1.8527839702879827, variance=1.7879947400121508, skewness=-0.6524797860367648, kurtosis=-1.4999999999999996)
[-3.357 -0.934 -1.214] DescribeResult(nobs=3, minmax=(-3.3574545036338255, -0.9339834531856035), mean=-1.8352140397957912, variance=1.7575429338010611, skewness=-0.671731426172235, kurtosis=-1.5000000000000002)
[-3.737 -1.045 -1.331] DescribeResult(nobs=3, minmax=(-3.7365216618110684, -1.045447417998383), mean=-2.0376020698029214, variance=2.1851076569537624, skewness=-0.6775710245213931, kurtosis=-1.5000000000000004)


In [38]:
labels

array([  7.446,  10.58 ,   6.611, ...,  -1.805,  -6.822,  -1.579])

In [29]:
summed_outliers_per_weight(weight_df, observer, n_metrics)

AttributeError: 'DataFrame' object has no attribute 'observer'

In [ ]:
# Single DataFrame of all features against known good
var_weights = target_weights.apply(lambda s: s/target_weights.CombinedTrust, axis=0).dropna()
known_good_features = \
    pd.concat([feature_extractor(s.reset_index(),var) for var,s  in var_weights.iteritems()],
              keys=var_weights.keys(), names=['var','metric'])

These results handily confirm that there is a 'signature' of badmouthing as RandomFlatWalk was incorrectly configured. 

Need to: 
1. Perform multi-run tolerance analysis of metrics (i.e. turn the below into a boxplot)
2. Perform cross correlation analysis on metrics across runs/behaviours (what metrics are redundant)

In [ ]:
_=known_good_features.unstack().plot(kind='bar')

In [ ]:
_=known_good_features.unstack().boxplot()

In [20]:
import operator

def target_weight_feature_extractor(target_weights):
    known_good_features_d = {}
    for basekey in target_weights.keys(): # Parallelisable
        print basekey
        # Single DataFrame of all features against one behaviour
        var_weights = target_weights.apply(lambda s: s/target_weights[basekey], axis=0).dropna()
        known_good_features_d[basekey] = \
            pd.concat([feature_extractor(s.reset_index(),var) for var,s  in var_weights.iteritems()],
                      keys=var_weights.keys(), names=['var','metric'])
            
    return known_good_features_d


def dataframe_weight_filter(df, keys):
    indexes = [(df.index.get_level_values(k)==0.0) for k in keys]
    return df.loc[reduce(operator.and_,indexes)]

phys_keys = ['INDD','INHD','Speed']
comm_keys = ['ADelay','ARXP','ATXP','RXThroughput','TXThroughput','PLR']

In [21]:
#Comms Only Weights
comms_target_weights=dataframe_weight_filter(target_weights,phys_keys)
comms_target_weights.reset_index(level=phys_keys, drop=True, inplace=True)
comms_features_d = target_weight_feature_extractor(comms_target_weights)
for var,feat in comms_features_d.iteritems():
    feat.unstack().plot(kind='bar', title=var)

NameError: name 'target_weights' is not defined

In [ ]:
#Phys Only Weights
phys_target_weights=dataframe_weight_filter(target_weights,comm_keys)
phys_target_weights.reset_index(level=comm_keys, drop=True, inplace=True)
phys_features_d = target_weight_feature_extractor(phys_target_weights)
for var,feat in phys_features_d.iteritems():
    feat.unstack().plot(kind='bar', title=var)